In [95]:
# License: BSD
# Author: Tianshuo Zhou

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
# import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
# plt.ion()   # interactive mode

In [2]:
from torch.utils.data import DataLoader
from sklearn.model_selection import  train_test_split

In [3]:
from datasets import SvdDataset

In [4]:
path_data = 'data/flowers.npy'
batch_size = 256
workers = 16
image_size = 224

triples = np.load(path_data)
print('triples: ')
print(triples[:10])
print(triples[-10:])

triples: 
[['data/flowers/daisy/100080576_f52e8ee070_n.jpg' '0']
 ['data/flowers/daisy/10140303196_b88d3d6cec.jpg' '0']
 ['data/flowers/daisy/10172379554_b296050f82_n.jpg' '0']
 ['data/flowers/daisy/10172567486_2748826a8b.jpg' '0']
 ['data/flowers/daisy/10172636503_21bededa75_n.jpg' '0']
 ['data/flowers/daisy/102841525_bd6628ae3c.jpg' '0']
 ['data/flowers/daisy/10300722094_28fa978807_n.jpg' '0']
 ['data/flowers/daisy/1031799732_e7f4008c03.jpg' '0']
 ['data/flowers/daisy/10391248763_1d16681106_n.jpg' '0']
 ['data/flowers/daisy/10437754174_22ec990b77_m.jpg' '0']]
[['data/flowers/tulip/9048307967_40a164a459_m.jpg' '4']
 ['data/flowers/tulip/924782410_94ed7913ca_m.jpg' '4']
 ['data/flowers/tulip/9378657435_89fabf13c9_n.jpg' '4']
 ['data/flowers/tulip/9444202147_405290415b_n.jpg' '4']
 ['data/flowers/tulip/9446982168_06c4d71da3_n.jpg' '4']
 ['data/flowers/tulip/9831362123_5aac525a99_n.jpg' '4']
 ['data/flowers/tulip/9870557734_88eb3b9e3b_n.jpg' '4']
 ['data/flowers/tulip/9947374414_fdf1d086

In [5]:
triples_train, triples_test = train_test_split(triples, test_size=3000, random_state=2)

In [6]:
len(triples_train), len(triples_test)

(1317, 3000)

In [16]:
transform_compose=transforms.Compose([
                           transforms.Resize(image_size),
                           transforms.CenterCrop(image_size),
                           # transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ])

dataset_train = SvdDataset(files_list=triples_train[:,0], labels_list=triples_train[:, 1], transform=transform_compose, files_list_svd=triples_train[:, 0])
# Create the dataloader
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=workers)
print(f'len(dataloader_train): {len(dataloader_train)}, len(dataloader_train): {len(dataloader_train)}')
for i, batch in enumerate(dataloader_train):
    images, labels = batch
    print(i, images.shape, labels.shape)

dataset_test = SvdDataset(files_list=triples_test[:,0], labels_list=triples_test[:, 1], transform=transform_compose, files_list_svd=triples_test[:, 0])
# Create the dataloader
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers=workers)
print(f'len(dataset_test): {len(dataset_test)}, len(dataloader_test): {len(dataloader_test)}')

for i, batch in enumerate(dataloader_test):
    images, labels = batch
    print(i, images.shape, labels.shape)

len(dataloader_train): 6, len(dataloader_train): 6
0 torch.Size([256, 3, 224, 224]) torch.Size([256])
1 torch.Size([256, 3, 224, 224]) torch.Size([256])
2 torch.Size([256, 3, 224, 224]) torch.Size([256])
3 torch.Size([256, 3, 224, 224]) torch.Size([256])
4 torch.Size([256, 3, 224, 224]) torch.Size([256])
5 torch.Size([37, 3, 224, 224]) torch.Size([37])
len(dataset_test): 3000, len(dataloader_test): 12
0 torch.Size([256, 3, 224, 224]) torch.Size([256])
1 torch.Size([256, 3, 224, 224]) torch.Size([256])
2 torch.Size([256, 3, 224, 224]) torch.Size([256])
3 torch.Size([256, 3, 224, 224]) torch.Size([256])
4 torch.Size([256, 3, 224, 224]) torch.Size([256])
5 torch.Size([256, 3, 224, 224]) torch.Size([256])
6 torch.Size([256, 3, 224, 224]) torch.Size([256])
7 torch.Size([256, 3, 224, 224]) torch.Size([256])
8 torch.Size([256, 3, 224, 224]) torch.Size([256])
9 torch.Size([256, 3, 224, 224]) torch.Size([256])
10 torch.Size([256, 3, 224, 224]) torch.Size([256])
11 torch.Size([184, 3, 224, 224])

In [7]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cuda', index=1)

In [9]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=25, scheduler=None):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            # with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize only if in training phase
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        if scheduler is not None:
            scheduler.step()

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'epoch: {epoch: 2d}, loss: {epoch_loss:.4f}, acc: {epoch_acc:.4f}')
    return model

In [10]:
def eval_model(model, dataloader):
    model.eval()
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'Valid Acc: {epoch_acc:.4f}')
    return epoch_acc

In [17]:
num_class = len(dataset_train.class_labels_dict)

In [18]:
num_class

5

In [15]:
transform_compose=transforms.Compose([
                           transforms.Resize(image_size),
                           transforms.CenterCrop(image_size),
                           transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ])

transform_compose_test=transforms.Compose([
                           transforms.Resize(image_size),
                           transforms.CenterCrop(image_size),
                           # transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ])

In [59]:
dataset_train = SvdDataset(files_list=triples_train[:,0], labels_list=triples_train[:, 1], transform=transform_compose, files_list_svd=triples_train[:, 0])
# Create the dataloader
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=workers)
print(f'len(dataloader_train): {len(dataloader_train)}, len(dataloader_train): {len(dataloader_train)}')
for i, batch in enumerate(dataloader_train):
    images, labels = batch
    print(i, images.shape, labels.shape)

dataset_test = SvdDataset(files_list=triples_test[:,0], labels_list=triples_test[:, 1], transform=transform_compose_test, files_list_svd=triples_test[:, 0])
# Create the dataloader
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers=workers)
print(f'len(dataset_test): {len(dataset_test)}, len(dataloader_test): {len(dataloader_test)}')

for i, batch in enumerate(dataloader_test):
    images, labels = batch
    print(i, images.shape, labels.shape)

len(dataloader_train): 6, len(dataloader_train): 6
0 torch.Size([256, 3, 224, 224]) torch.Size([256])
1 torch.Size([256, 3, 224, 224]) torch.Size([256])
2 torch.Size([256, 3, 224, 224]) torch.Size([256])
3 torch.Size([256, 3, 224, 224]) torch.Size([256])
4 torch.Size([256, 3, 224, 224]) torch.Size([256])
5 torch.Size([37, 3, 224, 224]) torch.Size([37])
len(dataset_test): 3000, len(dataloader_test): 12
0 torch.Size([256, 3, 224, 224]) torch.Size([256])
1 torch.Size([256, 3, 224, 224]) torch.Size([256])
2 torch.Size([256, 3, 224, 224]) torch.Size([256])
3 torch.Size([256, 3, 224, 224]) torch.Size([256])
4 torch.Size([256, 3, 224, 224]) torch.Size([256])
5 torch.Size([256, 3, 224, 224]) torch.Size([256])
6 torch.Size([256, 3, 224, 224]) torch.Size([256])
7 torch.Size([256, 3, 224, 224]) torch.Size([256])
8 torch.Size([256, 3, 224, 224]) torch.Size([256])
9 torch.Size([256, 3, 224, 224]) torch.Size([256])
10 torch.Size([256, 3, 224, 224]) torch.Size([256])
11 torch.Size([184, 3, 224, 224])

In [57]:
def train_eval_svd(tranfm_train, train_size=2000, train_epochs=20):
    test_size = len(triples) - train_size
    triples_train, triples_test = train_test_split(triples, test_size=test_size, random_state=2)
    files_svd_train = [file.replace('flowers', 'flowers_svd40') for file in triples_train[:, 0]]
    print(f'files_svd_train: {files_svd_train[:2]}')
    torch.manual_seed(2)
    dataset_train = SvdDataset(files_list=triples_train[:,0], labels_list=triples_train[:, 1], transform=tranfm_train, files_list_svd=files_svd_train)
    # Create the dataloader
    dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=workers)
    print(f'len(dataloader_train): {len(dataset_train)}, len(dataloader_train): {len(dataloader_train)}')
    # for i, batch in enumerate(dataloader_train):
    #     images, labels = batch
    #     print(i, images.shape, labels.shape)

    dataset_test = SvdDataset(files_list=triples_test[:,0], labels_list=triples_test[:, 1], transform=transform_compose_test, files_list_svd=triples_test[:, 0])
    # Create the dataloader
    dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers=workers)
    print(f'len(dataset_test): {len(dataset_test)}, len(dataloader_test): {len(dataloader_test)}')

#     for i, batch in enumerate(dataloader_test):
#         images, labels = batch
#         print(i, images.shape, labels.shape)
    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    # Here the size of each output sample is set to 2.
    # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
    model_ft.fc = nn.Linear(num_ftrs, num_class)
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.005, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
    model_train = train_model(model_ft, dataloader_train, criterion, optimizer_ft, num_epochs=train_epochs, scheduler=exp_lr_scheduler)
    acc_eval = eval_model(model_train, dataloader_test)
    print(f'acc_val: {acc_eval: .4f}')
    return acc_eval

In [58]:
def train_eval(tranfm_train, train_size=2000, train_epochs=20):
    test_size = len(triples) - train_size
    triples_train, triples_test = train_test_split(triples, test_size=test_size, random_state=2)
    dataset_train = SvdDataset(files_list=triples_train[:,0], labels_list=triples_train[:, 1], transform=tranfm_train, files_list_svd=triples_train[:, 0])
    # Create the dataloader
    torch.manual_seed(2)
    dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=workers)
    print(f'len(dataloader_train): {len(dataset_train)}, len(dataloader_train): {len(dataloader_train)}')
    # for i, batch in enumerate(dataloader_train):
    #     images, labels = batch
    #     print(i, images.shape, labels.shape)

    dataset_test = SvdDataset(files_list=triples_test[:,0], labels_list=triples_test[:, 1], transform=transform_compose_test, files_list_svd=triples_test[:, 0])
    # Create the dataloader
    dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers=workers)
    print(f'len(dataset_test): {len(dataset_test)}, len(dataloader_test): {len(dataloader_test)}')

#     for i, batch in enumerate(dataloader_test):
#         images, labels = batch
#         print(i, images.shape, labels.shape)
    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    # Here the size of each output sample is set to 2.
    # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
    model_ft.fc = nn.Linear(num_ftrs, num_class)
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.005, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
    model_train = train_model(model_ft, dataloader_train, criterion, optimizer_ft, num_epochs=train_epochs, scheduler=exp_lr_scheduler)
    acc_eval = eval_model(model_train, dataloader_test)
    print(f'acc_val: {acc_eval: .4f}')
    return acc_eval

In [60]:
batch_size=128
transform_compose=transforms.Compose([
                           transforms.Resize(image_size),
                           transforms.CenterCrop(image_size),
                           # transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ])

In [62]:
results = []
results_svd =[]
for size in [500, 1000, 1500, 2000, 2500, 3000]:
    acc = train_eval(transform_compose, train_size=size, train_epochs=10)
    acc_svd = train_eval_svd(transform_compose, train_size=size, train_epochs=10)
    results.append(acc)
    results_svd.append(acc_svd)
print('results: ', results)
print('results_svd: ', results_svd)

len(dataloader_train): 500, len(dataloader_train): 4
len(dataset_test): 3817, len(dataloader_test): 30
epoch:  0, loss: 1.7475, acc: 0.2120
epoch:  1, loss: 1.1253, acc: 0.6000
epoch:  2, loss: 0.6073, acc: 0.8640
epoch:  3, loss: 0.3683, acc: 0.9140
epoch:  4, loss: 0.2403, acc: 0.9440
epoch:  5, loss: 0.1454, acc: 0.9800
epoch:  6, loss: 0.0879, acc: 0.9880
epoch:  7, loss: 0.0628, acc: 0.9980
epoch:  8, loss: 0.0406, acc: 0.9980
epoch:  9, loss: 0.0236, acc: 0.9980
Valid Acc: 0.8779
acc_val:  0.8779
files_svd_train: ['data/flowers_svd40/dandelion/674407101_57676c40fb.jpg', 'data/flowers_svd40/tulip/16570062929_6c41c66387_n.jpg']
len(dataloader_train): 500, len(dataloader_train): 4
len(dataset_test): 3817, len(dataloader_test): 30
epoch:  0, loss: 1.7679, acc: 0.2080
epoch:  1, loss: 1.1827, acc: 0.5620
epoch:  2, loss: 0.6849, acc: 0.8280
epoch:  3, loss: 0.4400, acc: 0.8840
epoch:  4, loss: 0.2946, acc: 0.9340
epoch:  5, loss: 0.1956, acc: 0.9440
epoch:  6, loss: 0.1230, acc: 0.980

In [83]:
acc = train_eval(transform_compose, train_size=3500, train_epochs=10)
acc_svd = train_eval_svd(transform_compose, train_size=3500, train_epochs=10)

len(dataloader_train): 3500, len(dataloader_train): 28
len(dataset_test): 817, len(dataloader_test): 7
epoch:  0, loss: 0.7690, acc: 0.7194
epoch:  1, loss: 0.2225, acc: 0.9234
epoch:  2, loss: 0.1035, acc: 0.9726
epoch:  3, loss: 0.0551, acc: 0.9897
epoch:  4, loss: 0.0320, acc: 0.9954
epoch:  5, loss: 0.0170, acc: 0.9983
epoch:  6, loss: 0.0121, acc: 0.9994
epoch:  7, loss: 0.0105, acc: 0.9994
epoch:  8, loss: 0.0093, acc: 0.9980
epoch:  9, loss: 0.0082, acc: 0.9989
Valid Acc: 0.9461
acc_val:  0.9461
files_svd_train: ['data/flowers_svd40/tulip/485266837_671def8627.jpg', 'data/flowers_svd40/tulip/13979840624_28466cb3ec_n.jpg']
len(dataloader_train): 3500, len(dataloader_train): 28
len(dataset_test): 817, len(dataloader_test): 7
epoch:  0, loss: 0.8025, acc: 0.7069
epoch:  1, loss: 0.2656, acc: 0.9097
epoch:  2, loss: 0.1452, acc: 0.9540
epoch:  3, loss: 0.0878, acc: 0.9740
epoch:  4, loss: 0.0524, acc: 0.9874
epoch:  5, loss: 0.0293, acc: 0.9957
epoch:  6, loss: 0.0184, acc: 0.9983
ep

In [84]:
results.append(acc)
results_svd.append(acc_svd)
print('results: ', results)
print('results_svd: ', results_svd)

results:  [tensor(0.8779, device='cuda:1', dtype=torch.float64), tensor(0.8939, device='cuda:1', dtype=torch.float64), tensor(0.9049, device='cuda:1', dtype=torch.float64), tensor(0.9145, device='cuda:1', dtype=torch.float64), tensor(0.9263, device='cuda:1', dtype=torch.float64), tensor(0.9286, device='cuda:1', dtype=torch.float64), tensor(0.9461, device='cuda:1', dtype=torch.float64)]
results_svd:  [tensor(0.8800, device='cuda:1', dtype=torch.float64), tensor(0.9011, device='cuda:1', dtype=torch.float64), tensor(0.9134, device='cuda:1', dtype=torch.float64), tensor(0.9176, device='cuda:1', dtype=torch.float64), tensor(0.9268, device='cuda:1', dtype=torch.float64), tensor(0.9355, device='cuda:1', dtype=torch.float64), tensor(0.9510, device='cuda:1', dtype=torch.float64)]


In [64]:
transform_compose_flip=transforms.Compose([
                           transforms.Resize(image_size),
                           transforms.CenterCrop(image_size),
                           transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ])

In [66]:
results_flip = []
results_svd_flip =[]
for size in [500, 1000, 1500, 2000, 2500, 3000, 3500]:
    acc_flip = train_eval(transform_compose_flip, train_size=size, train_epochs=10)
    acc_svd_flip = train_eval_svd(transform_compose, train_size=size, train_epochs=10)
    results_flip.append(acc_flip)
    results_svd_flip.append(acc_svd_flip)
print('results_flip: ', results_flip)
print('results_svd_flip: ', results_svd_flip)

len(dataloader_train): 500, len(dataloader_train): 4
len(dataset_test): 3817, len(dataloader_test): 30
epoch:  0, loss: 1.7612, acc: 0.2180
epoch:  1, loss: 1.1480, acc: 0.5840
epoch:  2, loss: 0.6464, acc: 0.8340
epoch:  3, loss: 0.4138, acc: 0.8960
epoch:  4, loss: 0.2816, acc: 0.9240
epoch:  5, loss: 0.1837, acc: 0.9580
epoch:  6, loss: 0.1272, acc: 0.9660
epoch:  7, loss: 0.0995, acc: 0.9800
epoch:  8, loss: 0.0637, acc: 0.9920
epoch:  9, loss: 0.0412, acc: 0.9980
Valid Acc: 0.8790
acc_val:  0.8790
files_svd_train: ['data/flowers_svd40/dandelion/674407101_57676c40fb.jpg', 'data/flowers_svd40/tulip/16570062929_6c41c66387_n.jpg']
len(dataloader_train): 500, len(dataloader_train): 4
len(dataset_test): 3817, len(dataloader_test): 30
epoch:  0, loss: 1.7679, acc: 0.2080
epoch:  1, loss: 1.1827, acc: 0.5620
epoch:  2, loss: 0.6849, acc: 0.8280
epoch:  3, loss: 0.4400, acc: 0.8840
epoch:  4, loss: 0.2945, acc: 0.9340
epoch:  5, loss: 0.1957, acc: 0.9440
epoch:  6, loss: 0.1232, acc: 0.980

In [67]:
results_flip

[tensor(0.8790, device='cuda:1', dtype=torch.float64),
 tensor(0.8993, device='cuda:1', dtype=torch.float64),
 tensor(0.9081, device='cuda:1', dtype=torch.float64),
 tensor(0.9210, device='cuda:1', dtype=torch.float64),
 tensor(0.9285, device='cuda:1', dtype=torch.float64),
 tensor(0.9347, device='cuda:1', dtype=torch.float64),
 tensor(0.9474, device='cuda:1', dtype=torch.float64)]

In [68]:
results_svd_flip

[tensor(0.8805, device='cuda:1', dtype=torch.float64),
 tensor(0.9011, device='cuda:1', dtype=torch.float64),
 tensor(0.9141, device='cuda:1', dtype=torch.float64),
 tensor(0.9176, device='cuda:1', dtype=torch.float64),
 tensor(0.9268, device='cuda:1', dtype=torch.float64),
 tensor(0.9355, device='cuda:1', dtype=torch.float64),
 tensor(0.9523, device='cuda:1', dtype=torch.float64)]

In [85]:
results_all = []
results_all.append(np.array([x.cpu().item() for x in results]))
results_all.append(np.array([x.cpu().item() for x in results_svd]))
results_all.append(np.array([x.cpu().item() for x in results_flip]))
results_all.append(np.array([x.cpu().item() for x in results_svd_flip]))

In [86]:
np.save('data/results_classification.npy', np.array(results_all))

In [90]:
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.4f" % x))

In [91]:
results_all

[array([0.8779, 0.8939, 0.9049, 0.9145, 0.9263, 0.9286, 0.9461]),
 array([0.8800, 0.9011, 0.9134, 0.9176, 0.9268, 0.9355, 0.9510]),
 array([0.8790, 0.8993, 0.9081, 0.9210, 0.9285, 0.9347, 0.9474]),
 array([0.8805, 0.9011, 0.9141, 0.9176, 0.9268, 0.9355, 0.9523])]